In [20]:
# -*- encoding: utf8-*-
import requests
import time
import random
import json
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
#print(json.dumps(return_json, indent=2))

In [ ]:
def get_top_songs():
    top_songs = []

    headers = {'Referer': 'https://kma.kkbox.com/charts/yearly/newrelease?lang=tc&terr=tw',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
                'Content-Type': 'application/json'}
    url = 'https://kma.kkbox.com/charts/api/v1/yearly?category=297&lang=tc&limit=100&terr=tw&type=newrelease&year=2021'
    
    return_data = requests.get(url, headers=headers)
    return_json = json.loads(return_data.text)
    #print(json.dumps(return_json, indent=2))

    for song in return_json['data']['charts']['newrelease']:
        top_songs.append(re.split(' -| \(', song['song_name'])[0] + ' ' + song['artist_name'])
    
    return top_songs

In [ ]:
# 獲取樂譜頁面
def get_pu(song_name):

    url = 'https://www.google.com/search?q=' + song_name + ' 91譜'
    print(url)
    return_html = requests.get(url)
    soup = BeautifulSoup(return_html.text, 'html.parser')
    pu_url=''
    for tags in soup.find_all('a'):
        #print(tags.get('href'))
        if re.match(r'https://www.91pu.com.tw', tags.get('href')) != None:
            #print(tags.get('href'))
            pu_url = tags.get('href')
            pu_url = re.sub('.*https', 'https', pu_url)
            pu_url = re.sub('html.*', 'html', pu_url)
            break 
    #print(pu_url)
    if re.search(r'song', pu_url) == None:
        return ""
    return pu_url

In [ ]:
def get_chords_and_lyrics(pu_url):
    tot = []
    options = Options()
    options.add_argument("--disable-notifications")
    options.headless = True
    driver = webdriver.Edge(options=options)
    driver.implicitly_wait(3)

    driver.get(pu_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    tone = soup.find(id='tone_z').find_all('p')
    clean = re.compile('<.*?>')
    nbsp = re.compile(u'\xa0+')
    emsp = re.compile('\u2003')
    for tags in tone:
        output = tags.text
        output = re.sub(clean, '', output)
        output = re.sub(nbsp, ' ', output)
        output = re.sub(emsp, ' ', output)
        tot.append(output)
    driver.quit()
    return tot

In [26]:
def main():
    top_songs = get_top_songs()

    tots = []
    for i in tqdm(range(100)):
        try:
            pu_url = get_pu(top_songs[i])
            if pu_url == '':
                raise ValueError('Can\'t find pu_url!')
            tot = get_chords_and_lyrics(pu_url)
            tots.append(tot)
        except:
            tots.append(['no data'])
    
    # write file
    f = open('total4.txt', "w", encoding='UTF-8')
    num = 0
    for i in tots:
        f.write(top_songs[num]+"\n")
        for j in i:
            f.write(j + "\n")
        f.write("\n")
        num += 1
    f.close()

100%|██████████| 10/10 [04:56<00:00, 29.67s/it]


In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
# song_name='失重前幸福'

# pu_url = get_pu(song_name)
# tot = get_chords_and_lyrics(pu_url)
# print(tot)

In [ ]:

# f = open('total.txt', "a", encoding='UTF-8')
# f.write(song_name+"\n")
# for j in tot:
#     f.write(j + '\n')
# f.write("\n")
# f.close()

In [ ]:
#top_songs = get_top_songs()
# ff = open('song2.txt', 'w', encoding='utf-8')
# for i in range(50):
#     ff.write(top_songs[i] + ' ' + top_artists[i] + '\n')
# ff.close()

In [ ]:
# print(top_artists)